In [449]:
from selenium import webdriver # type: ignore
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [630]:
options = Options()
options.add_experimental_option("detach",True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)


In [632]:
# Navigate to the URL
driver.get("https://www.agoda.com/search?city=14524&locale=en-us&ckuid=e7f4cf43-4845-4006-a349-b6d47e17e003&prid=0&currency=MYR&correlationId=fc841ccc-c0cd-4af9-80a4-a0e3946a1b19&analyticsSessionId=1316014179889239648&pageTypeId=103&realLanguageId=1&languageId=1&origin=MY&stateCode=14&cid=1844104&userId=e7f4cf43-4845-4006-a349-b6d47e17e003&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=4&currencyCode=MYR&htmlLanguage=en-us&cultureInfoName=en-us&machineName=sg-pc-6i-acm-web-user-667d9f8649-dgh9p&trafficGroupId=1&trafficSubGroupId=84&aid=130589&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&browserFamily=Chrome&cdnDomain=agoda.net&checkIn=2024-10-01&checkOut=2024-10-03&rooms=1&adults=1&children=1&childages=8&priceCur=MYR&los=2&textToSearch=Kuala+Lumpur&travellerType=2&familyMode=off&ds=lNhAcKYCZ5PNdgTW&productType=2")


In [634]:
soup =BeautifulSoup(driver.page_source, "html.parser")
print(soup)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [636]:
listings =soup.find_all("div",class_="Gridstyled__GridStyled-sc-dfh2k0-0 kEJDDi PropertyCard__Container")
len(listings)


98

In [638]:
card=listings[93]
#Rating
Rating=card.find("div",class_="ReviewWithDemographic").text.replace("\n","")
Rating_data = Rating.split(" ")

#Price
Price=int(card.find("div", class_="Box-sc-kv6pi1-0 fgWmJn").text.replace("RM", "").strip().replace("\xa0", ""))
print(Rating)

8.4Excellent1,325 reviews


In [640]:
Place_box=soup.find_all("div",class_="Itemstyled__Item-sc-12uga7p-0 ewNxOO PropertyCard__Section PropertyCard__Section--propertyInfo withPackageBundle")
len(Place_box)

90

In [642]:
#Place Name
Name=card.find("h3").text.replace("\n","")
print(Name)


Manatidur@THE HUB SS2


In [644]:
#Date , check in and check out
Checkbox=soup.find("div",class_="Searchbox Searchbox--horizontal")
len(Checkbox)

1

In [646]:
#Place
Place=Checkbox.find("div",class_="SearchBoxTextDescription__title").text.replace("\n","")
print(Place)

Kuala Lumpur


In [650]:
#Check out and check In
url="https://www.agoda.com/search?city=14524&locale=en-us&ckuid=e7f4cf43-4845-4006-a349-b6d47e17e003&prid=0&currency=MYR&correlationId=fc841ccc-c0cd-4af9-80a4-a0e3946a1b19&analyticsSessionId=1316014179889239648&pageTypeId=103&realLanguageId=1&languageId=1&origin=MY&stateCode=14&cid=1844104&userId=e7f4cf43-4845-4006-a349-b6d47e17e003&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=4&currencyCode=MYR&htmlLanguage=en-us&cultureInfoName=en-us&machineName=sg-pc-6i-acm-web-user-667d9f8649-dgh9p&trafficGroupId=1&trafficSubGroupId=84&aid=130589&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&browserFamily=Chrome&cdnDomain=agoda.net&checkIn=2024-10-01&checkOut=2024-10-03&rooms=1&adults=1&children=1&childages=8&priceCur=MYR&los=2&textToSearch=Kuala+Lumpur&travellerType=2&familyMode=off&ds=lNhAcKYCZ5PNdgTW&productType=2"
date_regex = r"(\d{4}-\d{2}-\d{2})"
Check_In = re. search(r"checkIn=" + date_regex, url).group(1)
Check_Out = re. search(r"checkOut=" + date_regex, url).group(1)
print(Check_In)
print(Check_Out)

2024-10-01
2024-10-03


In [652]:
#Pax
pax=Checkbox.find("div",class_="SearchBoxTextDescription SearchBoxTextDescription--occupancy").text.replace("\n","")
Pax=pax.replace("1 room"," ")


In [654]:
def get_hotel_listings(card):
    
    #Name
    Name=card.find("h3").text.replace("\n","") 
    #Price
    price_element = card.find("div", class_="Box-sc-kv6pi1-0 fgWmJn")
    Price = int(price_element.text.replace("RM", "").strip().replace("\xa0", "")) if price_element else 0
    #Rating
    Rating=card.find("div",class_="ReviewWithDemographic").text.replace("\n","")
    hotel_listings = (Name,Price,Rating)
    return hotel_listings
    

In [676]:
hotels = []
listings =soup.find_all("div",class_="Gridstyled__GridStyled-sc-dfh2k0-0 kEJDDi PropertyCard__Container")

for card in listings:
    hotel_lists = get_hotel_listings(card)
    if hotel_lists:
        hotels.append(hotel_lists)

In [658]:
print(hotels)

[('Sky Suites KLCC by Autumn Suites Premium Stay', 160, '7.6Very good3,696 reviews'), ('Soho suites KLCC by Homestay', 0, '7.8Very good234 reviews'), ('Swiss Garden Residence Kuala Lumpur', 112, '7.7Very good10,073 reviews'), ('Eaton Residences KLCC by Luna', 0, '8.7Excellent1,242 reviews'), ('Tropicana The Residence By Klcc', 365, '8.6Excellent1,249 reviews'), ('Regalia Residence @ KLCC View Sky Pool by KC', 78, '7.6Very good619 reviews'), ('Arte Mont Kiara By Awesomestay', 117, '7.2Very good1,000 reviews'), ('Expressionz Professional Suites  Wodages ', 110, '8.2Excellent5,041 reviews'), ('Greystone One Bukit Ceylon ', 94, '7.9Very good1,565 reviews'), ('Casa Residency Apartment', 73, '8.1Excellent1,305 reviews'), ('Yemala Suites @ Vortex KLCC', 238, '8.1Excellent1,136 reviews'), ('Alanis & Core Suite Klia By Dreamscape', 90, '8.0Excellent4,011 reviews'), ('Legasi Kampung Baru Guesthouse', 122, '8.7Excellent4,175 reviews'), ('Eaton Residence KLCC @ infinity Pool View', 0, '8.8Excellen

In [674]:
#Web
Type="Agoda"
Duplicate_Data=[[pax,Check_In,Check_Out,Place,Type]]
Dummy_Data=pd.DataFrame(Duplicate_Data,columns=["Pax","Check In","Check Out","Place","Web"])
new_Duplicate_Data = Dummy_Data.reindex(index=Dummy_Data.index.repeat(len(listings))).reset_index(drop=True)
print(new_Duplicate_Data)

                      Pax    Check In   Check Out         Place    Web
0   1 adult,1 child1 room  2024-10-01  2024-10-03  Kuala Lumpur  Agoda
1   1 adult,1 child1 room  2024-10-01  2024-10-03  Kuala Lumpur  Agoda
2   1 adult,1 child1 room  2024-10-01  2024-10-03  Kuala Lumpur  Agoda
3   1 adult,1 child1 room  2024-10-01  2024-10-03  Kuala Lumpur  Agoda
4   1 adult,1 child1 room  2024-10-01  2024-10-03  Kuala Lumpur  Agoda
..                    ...         ...         ...           ...    ...
93  1 adult,1 child1 room  2024-10-01  2024-10-03  Kuala Lumpur  Agoda
94  1 adult,1 child1 room  2024-10-01  2024-10-03  Kuala Lumpur  Agoda
95  1 adult,1 child1 room  2024-10-01  2024-10-03  Kuala Lumpur  Agoda
96  1 adult,1 child1 room  2024-10-01  2024-10-03  Kuala Lumpur  Agoda
97  1 adult,1 child1 room  2024-10-01  2024-10-03  Kuala Lumpur  Agoda

[98 rows x 5 columns]


In [670]:
def remove_duplicate(string):
    new_string=string[:3]
    return new_string

In [678]:
Data=pd.DataFrame(hotels,columns=["Name Hotel","Price(Rm)","Ratings"])
Data["Ratings"]=Data["Ratings"].astype(str).apply(remove_duplicate)
print(Data)

                                           Name Hotel  Price(Rm) Ratings
0       Sky Suites KLCC by Autumn Suites Premium Stay        160     7.6
1                        Soho suites KLCC by Homestay          0     7.8
2                 Swiss Garden Residence Kuala Lumpur        112     7.7
3                       Eaton Residences KLCC by Luna          0     8.7
4                     Tropicana The Residence By Klcc        365     8.6
..                                                ...        ...     ...
93                              Manatidur@THE HUB SS2         88     8.4
94                  Axon Serviced Suites Kuala Lumpur          0     8.2
95  Pacific Home Petaling Jaya @ The Curve, 1 Utam...          0     7.8
96        Regalia Residence And Suites By Awesomestay        108     6.0
97     Crystal Suites at Axon Residence near Pavilion          0     8.7

[98 rows x 3 columns]


In [680]:
Final_Data =new_Duplicate_Data.join(Data)
Final_Data

,Pax,Check In,Check Out,Place,Web,Name Hotel,Price(Rm),Ratings
0,"1 adult,1 child1 room",2024-10-01,2024-10-03,Kuala Lumpur,Agoda,Sky Suites KLCC by Autumn Suites Premium Stay,160,7.6
1,"1 adult,1 child1 room",2024-10-01,2024-10-03,Kuala Lumpur,Agoda,Soho suites KLCC by Homestay,0,7.8
2,"1 adult,1 child1 room",2024-10-01,2024-10-03,Kuala Lumpur,Agoda,Swiss Garden Residence Kuala Lumpur,112,7.7
3,"1 adult,1 child1 room",2024-10-01,2024-10-03,Kuala Lumpur,Agoda,Eaton Residences KLCC by Luna,0,8.7
4,"1 adult,1 child1 room",2024-10-01,2024-10-03,Kuala Lumpur,Agoda,Tropicana The Residence By Klcc,365,8.6
...,...,...,...,...,...,...,...,...
93,"1 adult,1 child1 room",2024-10-01,2024-10-03,Kuala Lumpur,Agoda,Manatidur@THE HUB SS2,88,8.4
94,"1 adult,1 child1 room",2024-10-01,2024-10-03,Kuala Lumpur,Agoda,Axon Serviced Suites Kuala Lumpur,0,8.2
95,"1 adult,1 child1 room",2024-10-01,2024-10-03,Kuala Lumpur,Agoda,"Pacific Home Petaling Jaya @ The Curve, 1 Utam...",0,7.8
96,"1 adult,1 child1 room",2024-10-01,2024-10-03,Kuala Lumpur,Agoda,Regalia Residence And Suites By Awesomestay,108,6.0


In [682]:
Final_Data.to_csv(f"D:/WebScraping/Agoda_{Check_In}_{Check_Out}_{Place}.csv",index=False)